Work straight from matrix equations

In [1]:
import numpy as np
import pandas as pd
from cmm_error_map import data_cmpts as dc
from cmm_error_map import design_matrix_linear_fixed as design

In [2]:
def linear_dependency(xyz, model_params):
    x,y,z = xyz
    ld_dict = {
        "Txx": x,
        "Txy": x,
        "Txz": x,
        "Tyx": y,
        "Tyy": y,
        "Tyz": y,
        "Tzx": z,
        "Tzy": z,
        "Tzz": z,
        "Rxx": x,
        "Rxy": x,
        "Rxz": x,
        "Ryx": y,
        "Ryy": y,
        "Ryz": y,
        "Rzx": z,
        "Rzy": z,
        "Rzz": z,
        "Wxy": x,
        "Wxz": y,
        "Wyz": z,
    }
    pl = {key: model_params[key]* ld_dict[key] for key in model_params}
    return pl

In [3]:
def linear_model_matrix(xyz, xyzt, model_params):
    pl = linear_dependency(xyz, model_params)
    # rotation
    rxl = np.array([[1.,       pl['Rxz'], -pl['Rxy']], 
                [-pl['Rxz'],   1.,      pl['Rxx']],
                [ pl['Rxy'],-pl['Rxx'],     1.   ]])
    ryl = np.array([[1.,       pl['Ryz'], -pl['Ryy']], 
                [-pl['Ryz'],   1.,      pl['Ryx']],
                [ pl['Ryy'],-pl['Ryx'],     1.   ]])
    rzl = np.array([[1.,       pl['Rzz'], -pl['Rzy']], 
                [-pl['Rzz'],   1.,      pl['Rzx']],
                [ pl['Rzy'],-pl['Rzx'],     1.   ]])
    # inverse rotation
    inv_ryl = np.linalg.inv(ryl)
    inv_rzl = np.linalg.inv(rzl)
    # translation
    x, y, z = xyz
    xl = np.array([x + pl['Txx'], pl['Txy'], pl['Txz'] ])
    yl = np.array([pl['Tyx'], y + pl['Tyy'], pl['Tyz'] ])
    zl = np.array([pl['Tzx'], pl['Tzy'], z + pl['Tzz'] ])
    w = rxl @ yl - rxl @ xl + rxl @ inv_ryl @ zl + rxl @ inv_ryl @ inv_rzl @ xyzt
    return w
    
    
    

In [4]:
xyz  = np.array([100.,50.,20.])
x,y,z = xyz
xyzt = np.array([0.,0.,0.])
xt,yt,zt = xyzt

In [51]:
# compare design and matrix functions
results = pd.DataFrame(columns= ['value', 'w0', 'w1', 'w2', 'xE', 'yE', 'zE'])
for key in dc.model_parameters_dict:
    mp = dc.model_parameters_dict.copy()
    mp[key] = 1e-4
    w1 = linear_model_matrix(xyz, xyzt, mp)
    xE, yE, zE = design.model_linear(x,y,z, list(mp.values()), xt, yt, zt)
    results.loc[key] = [mp[key], w1[0], w1[1], w1[2], xE, yE, zE ]



In [52]:
results

,value,w0,w1,w2,xE,yE,zE
Txx,0.0001,-100.010000,50.000000,20.0000,0.010,0.000,0.000
Txy,0.0001,-100.000000,49.990000,20.0000,0.000,-0.010,0.000
Txz,0.0001,-100.000000,50.000000,19.9900,0.000,0.000,-0.010
Tyx,0.0001,-99.995000,50.000000,20.0000,-0.005,0.000,0.000
Tyy,0.0001,-100.000000,50.005000,20.0000,0.000,0.005,0.000
Tyz,0.0001,-100.000000,50.000000,20.0050,0.000,0.000,-0.005
Tzx,0.0001,-99.998000,50.000000,20.0000,-0.002,0.000,0.000
Tzy,0.0001,-100.000000,50.002000,20.0000,0.000,-0.002,0.000
Tzz,0.0001,-100.000000,50.000000,20.0020,0.000,0.000,0.002
Rxx,0.0001,-100.000000,50.200000,19.5000,0.000,0.200,-0.500


In [53]:
results.to_clipboard()

In [17]:
w = linear_model_matrix(xyz, xyzt, dc.model_parameters_test)
w

array([-100.0013901 ,   49.99990706,   19.99980922])

In [18]:
w = linear_model_matrix(xyz, xyzt, dc.model_parameters_dict)
w

array([-100.,   50.,   20.])

In [34]:
dc.model_parameters_dict

{'Txx': 0.0,
 'Txy': 0.0,
 'Txz': 0.0,
 'Tyx': 0.0,
 'Tyy': 0.0,
 'Tyz': 0.0,
 'Tzx': 0.0,
 'Tzy': 0.0,
 'Tzz': 0.0,
 'Rxx': 0.0,
 'Rxy': 0.0,
 'Rxz': 0.0,
 'Ryx': 0.0,
 'Ryy': 0.0,
 'Ryz': 0.0,
 'Rzx': 0.0,
 'Rzy': 0.0,
 'Rzz': 0.0,
 'Wxy': 0.0,
 'Wxz': 0.0,
 'Wyz': 0.0}

In [22]:
model_params = dc.model_parameters_dict

pl= linear_dependency(xyz, model_params)
# rotation
rxl = np.array([[1.,       pl['Rxz'], -pl['Rxy']], 
            [-pl['Rxz'],   1.,      pl['Rxx']],
            [ pl['Rxy'],-pl['Rxx'],     1.   ]])
ryl = np.array([[1.,       pl['Ryz'], -pl['Ryy']], 
            [-pl['Ryz'],   1.,      pl['Ryx']],
            [ pl['Ryy'],-pl['Ryx'],     1.   ]])
rzl = np.array([[1.,       pl['Rzz'], -pl['Rzy']], 
            [-pl['Rzz'],   1.,      pl['Rzx']],
            [ pl['Rzy'],-pl['Rzx'],     1.   ]])
# inverse rotation
inv_ryl = np.linalg.inv(ryl)
inv_rzl = np.linalg.inv(rzl)
# translation
x,y,z =xyz
xl = np.array([x + pl['Txx'], pl['Txy'], pl['Txz'] ])
yl = np.array([pl['Tyx'], y + pl['Tyy'], pl['Tyz'] ])
zl = np.array([pl['Tzx'], pl['Tzy'], z + pl['Tzz'] ])
w = rxl @ yl - rxl @ xl + rxl @ inv_ryl @ zl + rxl @ inv_ryl @ inv_rzl @ xyzt

In [23]:
w

array([-100.,   50.,   20.])

In [24]:
rxl

array([[ 1.,  0., -0.],
       [-0.,  1.,  0.],
       [ 0., -0.,  1.]])

In [25]:
inv_ryl

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

In [26]:
inv_rzl

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])